In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from IPython.display import display, Markdown, Latex
import pandas as pd
from uncertainties import ufloat
from uncertainties import unumpy as unp
from scipy import constants as const 

<center>Anna Lechner, Paula Erhard</center>

# PW7 - Wärme und Wärmetransport

## Wärmetransport

### Versuchsaufbau und Durchführung 
Für zwei Betriebsbedingungen der Glühlampe wird die Spannung der Photozelle bei verschiedenen Abständen  gemessen. Da der Photostrom proportional zur Beleuchtungsstärke ist, folgt auch die Spannung $U \propto E$. Die Messwerte werden als Spannung $U$ gegen $(1/r^{2})$ geplottet, da die Beleuchtungsstärke mit dem Abstandsgesetz skaliert. Aus den Steigungen der beiden Ausgleichsgeraden erhält man das Verhältnis der Lichtstärken. Mit den gemessenen Leistungen der beiden Heizspannungen werden schließlich die Strahlungstemperaturen bestimmt.

### Wichtige Formeln

Beleuchtungsstärke:
$$E = \frac{CI}{r^{2}}$$

Verhältnis der Lichtstärken:
$$\frac{I_{1}}{I_{2}} = \frac{a_{1}}{a_{2}}$$

Stefan–Boltzmann-Beziehung:
$$\frac{P_{1}}{P_{2}} = \frac{T_{1}^{4}}{T_{2}^{4}}$$

Temperaturbestimmung:
$$\ln\!\left(\frac{I_{1}}{I_{2}}\right)
= \frac{1}{T_{1}}
\left[\left(\frac{P_{1}}{P_{2}}\right)^{1/4} - 1\right]
\left(\frac{ch}{k\lambda}\right)$$



### Verwendete Geräte und Unsicherheiten

### Ergebnisse 

In [4]:

_lambda = 580   #Wellenlänge Licht [nm]

#Betriebsbedingung 4V
U_Lampe_4 = 4     #Betriebsspannung Lampe; einstellen [V] #was sind die Unsicherheiten?? 
I_Lampe_4 =       #Betriebsstrom Lampe; ablesen [A]

U_4 = np.array([])  #messen bei Photozelle; proportional Strom und somit auch Beleuchtungsstärke [V]
U_rel_4 = ((U_4*  )/U_4)*100   #relative Unsicherheit; vom Datenblatt entnehmen [%]
uU_4 = unp.uarray(U_4, (U_4*U_rel_4)) #Spannung an Photozelle in uarray dargestellt [V]

#Betriebsbedingung 6V
U_Lampe_6 = 6     #Betriebsspannung Lampe; einstellen [V]
I_Lampe_6 =       #Betriebsstrom Lampe; ablesen [A]

U_6 = np.array([])  #messen bei Photozelle; proportional Strom und somit auch Beleuchtungsstärke [V]
U_rel_6 = ((U_6*  )/U_6)*100   #relative Unsicherheit; vom Datenblatt entnehmen [%]
uU_6 = unp.uarray(U_6, (U_6*U_rel_6)) #Spannung an Photozelle in uarray dargestellt [V]

#Abstände (für beide gleich)
au = 1      #ableseunsicherheit bei Distanzmessung [mm]
r = unp.uarray([], au)    #Abstand zw Lampe und Photozelle [mm] #WICHTIG: bei beiden Betriebsbedingungen gleiche Abstände und äquidistant!!; nicht weniger als 500mm


#lineare regressionen für Berechnung des Intensitätsverhältnis
a_4, intercept_4, r_4, _, std_4 = linregress((1/r**2), U_4)
a_6, intercept_6, r_6, _, std_6 = linregress((1/r**2), U_6)

I_ratio = a_4/a_6   #welche unsicherheiten? nochanschauen

#elektrische Leistung
P_1 = U_Lampe_4 * I_Lampe_4     #[W]
P_2 = U_Lampe_6 * I_Lampe_6     #[W]
P_ratio = P_1/P_2
#Strahlungstemperaturen
faktor = (const.c * const.h)/(const.Boltzmann *_lambda) #c*h/k*lamda konstante in der formel

T_1 = (1/(np.log(I_ratio))) * ((P_ratio**(1/4))-1) * faktor
T_2 = (((T_1**4)*P_2)/P_1)**(1/4) 

# nur noch alles printen und schön darstellen
data = {'r [mm]': r, 'U_1 [V]': uU_4, 'U_2 [V]': uU_6}
df = pd.DataFrame(data)
display(df.style.set_caption("Messwerte der Spannungen und Abstände"))


display(Markdown(f"$(\frac{I_1}{I_2})={I_ratio:.2f}"))

display(Markdown(f"$P_1 = {P_1:.2f} [W]"))
display(Markdown(f"$P_2 = {P_2:.2f} [W]"))

display(Markdown(f"$T_1 = {T_1:.2f} [K]"))
display(Markdown(f"$T_2 = {T_2:.2f} [K]"))

plt.figure()
plt.errorbar((1/r**2), U_4, yerr=unp.std_devs(uU_4), fmt='o', capsize=3, label="Messwerte Betriebsspannung 4V", color="orange")
plt.errorbar((1/r**2), U_6, yerr=unp.std_devs(uU_6), fmt='o', capsize=3, label="Messwerte Betriebsspannung 6V", color="blue")
plt.plot((1/r**2), (1/r**2)*a_4+intercept_4, label=rf"lineare Regressionsgerade; r²={r_4**2:.3f}")
plt.plot((1/r**2), (1/r**2)*a_6+intercept_6, label=rf"lineare Regressionsgerade; r²={r_6**2:.3f}")
plt.xlabel("1/(Abstandsquadrat) [1/mm^2]")
plt.ylabel("Spannung U [V]")
plt.title("Beleuchtungsstärke in Zusammenhang mit Abstand")
plt.legend()
plt.grid(True)
plt.show()

display(Markdown(f"Die Steigung der 1. linearen Regrssionsgeraden ist: $a_1 = {a_4:.2f}"))
display(Markdown(f"Die Steigung der 2. linearen Regrssionsgeraden ist: $a_2 = {a_6:.2f}"))





SyntaxError: invalid syntax (4041258981.py, line 5)

### Diskussion

## Wärmeleitfähigkeit von Metallen

### Versuchsaufbau und Durchführung
Für die Bestimmung der Wärmeleitfähigkeit wird ein Aluminiumstab an einem Ende beheizt und am anderen über einen Kupferblock gekühlt, bis ein stationärer Wärmestrom entsteht. Mit der Wärmebildkamera wird ein Temperaturprofil aufgenommen, aus dem der lineare Temperaturgradient $\Delta T / \Delta x$ bestimmt wird. Die Wärmeleitung folgt
$$\frac{\Delta Q}{\Delta t} = -\lambda A\,\frac{\Delta T}{\Delta x}$$
und die zugeführte Heizleistung lautet
$$P = \frac{U^{2}}{R}.$$
Aus Wärmestrom, Querschnittsfläche und Temperaturgradient wird schließlich die Wärmeleitfähigkeit $\lambda$ berechnet.

### Verwendetet Geräte und Unsicherheiten

### Ergebnisse

In [1]:
# Wärmeleitfähigkeit

a = unp.ufloat( , 1)     #Seitenlänge 1 der Querschnittsfläche [mm]
b = unp.ufloat( , 1)     #Seitenlänge 2 der Querschnittsfläche [mm]
l = unp.ufloat( , 1)     #länge auf der der Temperaunterschied stattfindet [mm]
A = a*b                  #Querschnittsfläche [mm²]
U = 14                   #[V]
R = 330                  #[Ohm]
K = U**2/R               #Konstante (dQ/dt) ermitteln aus Spannung und Widerstand (anscheinend)

df = pd.read_csv("", sep='\t', decimal=',', engine='python')
px = df[""].to_numpy()   #pixel aus csv file
t = df[""].to_numpy()    #temperatur aus csv file
px_zahl =                #höchste Zahl aus csv file eintragen

slope, intercept, r, _, std = linregress(l/px, t)               #fitten für temperaturanstieg
steigung = unp.ufloat(slope, std)

lambda_ = -(K/(steigung*A))     #Wärmeleitfähigkeit; KA ob die Formel stimmt (von emmi geklaut) [J/msK]

display(Markdown(f"$ \lambda = {lambda_:.2f} [J/msK]$"))

#plot
plt.figure()
plt.plot((l/px_zahl)*px, t, 'o', markersize=2, label="Temperatur pro Pixel aus Wärmebildkamera")
plt.plot((l/px_zahl)*px, slope*((l/px_zahl)*px)+intercept, 'r-', label=rf"Temperarturanstieg \n r² = {r**2:.3f}")
plt.xlabel("Abstand [cm]")
plt.ylabel("Temperatur [K]")
plt.title("Temperaturänderung")
plt.legend()
plt.grid(True)
plt.show()

display(Markdown(f"Die Steigung der linearen Regressionsfunktion ist {steigung:.2f}"))


#Bestimmung ALuminium / Duraluminium
Literaturwert = 235     #Aluminium (falls notwendig ändern auf 170 --> Duraluminium) [J/msK]

display(Markdown(f"Die ermittelte Wärmeleitfähigkeit von $\lambda = {lambda_:.2f} [J/msK]$ entspricht der von Aluminium/Duraluminium mit Literaturwert $\lambda = {Literaturwert} [J/msK$."))

SyntaxError: invalid syntax (2653867678.py, line 3)

### Diskussion